In [1]:
import pandas as pd

df = pd.read_csv('images_2.csv')
df

,image,label
0,ea7b6656-3f84-4eb3-9099-23e623fc1018,Top
1,ea2ffd4d-9b25-4ca8-9dc2-bd27f1cc59fa,Top
2,3b86d877-2b9e-4c8b-a6a2-1d87513309d0,Shoes
3,5d3a1404-697f-479f-9090-c1ecd0413d27,Bottoms
4,b0c03127-9dfb-4573-8934-1958396937bf,Top
...,...,...
4672,3855ea22-5e7f-411f-b1fa-6db27a676c06,Shoes
4673,dfd4079d-967b-4b3e-8574-fbac11b58103,Bottoms
4674,5379356a-40ee-4890-b416-2336a7d84061,Bottoms
4675,65507fb8-3456-4c15-b53e-d1b03bf71a59,Shoes


# Encode labels, take sample of data into train/valid/test

In [2]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels = le.fit_transform(df.label)
labels

array([3, 3, 2, ..., 0, 2, 0])

In [3]:
images = df.iloc[:, 0]

In [4]:
from sklearn.model_selection import StratifiedShuffleSplit

ss = StratifiedShuffleSplit(n_splits=1, test_size=0.1)
for _, group_index in ss.split(images, labels):
    group_x = images[group_index].to_numpy()
    group_y = labels[group_index]

In [5]:
ss = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
for train_index, test_index in ss.split(group_x, group_y):
    train_x, test_x = group_x[train_index], group_x[test_index]
    train_y, test_y = group_y[train_index], group_y[test_index]

In [6]:
ss = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
for train_index, valid_index in ss.split(train_x, train_y):
    train_x, valid_x = train_x[train_index], train_x[valid_index]
    train_y, valid_y = train_y[train_index], train_y[valid_index]

# Get all images loaded in from sample and convert to tensor

In [7]:
import numpy as np
from skimage.io import imread
from skimage.transform import resize

def get_images(path_list):
    train_img = []
    for filename in path_list:
        path = 'images_compressed/' + filename + '.jpg'
        img = imread(path, as_gray=True)
        img = resize(img, (100, 100))
        img /= 255.0
        img = img.astype('float32')
        train_img.append(img)
    return np.asarray(train_img)

In [8]:
train_x_images = get_images(train_x)

In [9]:
train_x_images.shape

(299, 100, 100)

In [10]:
valid_x_images = get_images(valid_x)
test_x_images = get_images(test_x)

In [17]:
import torch

train_x_torch = torch.from_numpy(train_x_images).unsqueeze(dim=1)
valid_x_torch = torch.from_numpy(valid_x_images).unsqueeze(dim=1)
test_x_torch = torch.from_numpy(test_x_images).unsqueeze(dim=1)

In [18]:
train_x_torch.shape

torch.Size([299, 1, 100, 100])

In [19]:
train_y_torch = torch.from_numpy(train_y)
valid_y_torch = torch.from_numpy(valid_y)
test_y_torch = torch.from_numpy(test_y)
train_y_torch.shape

torch.Size([299])

In [78]:
# use gpu if available
if torch.cuda.is_available():
    train_x_torch = train_x_torch.cuda()
    train_y_torch = train_y_torch.cuda()
    valid_x_torch = valid_x_torch.cuda()
    valid_y_torch = valid_y_torch.cuda()
    test_x_torch = test_x_torch.cuda()
    test_y_torch = test_y_torch.cuda()
    

In [79]:
train = list(zip(train_x_torch, train_y_torch.to(dtype=torch.long)))
test = list(zip(test_x_torch, test_y_torch.to(dtype=torch.long)))
valid = list(zip(valid_x_torch, valid_y_torch.to(dtype=torch.long)))

In [80]:
# put everything into a data loader because too much data for gpu

from torch.utils.data import DataLoader

trainloader = DataLoader(train, batch_size=16)
testloader = DataLoader(test, batch_size=16)
validloader = DataLoader(test, batch_size=16)

# Create CNN model

In [93]:
from torch import nn

# need to figure out what the numbers in Conv2d and other parts are / what to use for them

class ConvNN(nn.Module):
    def __init__(self):
        super(ConvNN, self).__init__()
        
        # two layers for this cnn
        self.cnn_layers = nn.Sequential (
            # Defining a 2D convolution layer
            nn.Conv2d(1, 32, kernel_size=9, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            nn.Conv2d(32, 32, kernel_size=9, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

    
        self.linear_layers = nn.Sequential(
            nn.Linear(32 * 20 * 20, 4)
        )

    
    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
#         print(x.shape)
        x = self.linear_layers(x)
        return x

In [97]:
from torch import optim

model = ConvNN()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

ConvNN(
  (cnn_layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(9, 9), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 32, kernel_size=(9, 9), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=12800, out_features=4, bias=True)
  )
)


In [98]:
# Training loop

epochs = 10
for epoch in range(epochs):
    train_loss = 0.
    valid_loss = 0.
    for images, labels in trainloader:
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    for images, labels in validloader:
        output = model(images)
        loss = criterion(output, labels)
        valid_loss += loss.item()
    print('At epoch', epoch, 'loss =', train_loss/len(trainloader), 'valid loss =', valid_loss/len(validloader))

At epoch 0 loss = 1.0829087621287297 valid loss = 0.9260255595048269
At epoch 1 loss = 0.706309520884564 valid loss = 0.8474856515725454
At epoch 2 loss = 0.5539311750939018 valid loss = 0.8033259212970734
At epoch 3 loss = 0.4231521879371844 valid loss = 0.749553918838501
At epoch 4 loss = 0.31454309977983175 valid loss = 0.7115515718857447
At epoch 5 loss = 0.2376021954574083 valid loss = 0.6999213049809138
At epoch 6 loss = 0.18614927130310158 valid loss = 0.7748958617448807
At epoch 7 loss = 0.15331761931118212 valid loss = 0.8757546295722326
At epoch 8 loss = 0.12912801417865252 valid loss = 0.7970501432816187
At epoch 9 loss = 0.11077100646338965 valid loss = 0.644443134466807
